<a href="https://colab.research.google.com/github/steeve85ok/SIM_YAC_ALMEIDA_UPSE2023/blob/main/meteximpl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import numpy as np
import math

In [34]:
L=10000 #long reservorio
dx=2500
Tiempo=100000
DT=1
k=50 #Permeabilidad MD
por=0.2 #porosidad
vis=1 #centpoise
Beta=1 #asumir cte
CompTotal=1**-6 #psi-1
A=200000 #ft 2



In [35]:
Alpha=k/(vis*por*CompTotal)
Alpha
Eta=(Alpha*DT)/(dx**2)
Eta

4e-05

In [42]:
#Esquema Explícito con Eta
def Presdt(P0,P1,P2,Alpha):
  Pdt=P1+Eta*(P0-2*P1+P2)
  return(Pdt)

Pexplicita= np.zeros((Tiempo//DT,L//dx))




In [37]:
#Condiciones iniciales
Pyac=1000
Piz=2000
Pder=0
Pin=np.ones(L//dx)
Pres=np.zeros(L//dx)
Pin=Pin*Pyac
Pin[0]=Piz
Pin[len(Pin)-1]=Pder
Pin

array([2000., 1000., 1000.,    0.])

In [44]:
t=DT
Pexplicita[0]=Pin #primera fila de matriz se llena con valores iniciales
Pexplicita.shape
while t<Tiempo:
  for i in range(0,len(Pin)-1):
    if i==0:
      Pres[i]=Pin[0]
    elif i==len(Pin)-1:
      Pres[i]=Pin[len(Pin)-1]
    else:
      Pres[i]=Presdt(Pin[i-1],Pin[i],[i+1],Eta)
  Pexplicita[t//DT]=Pres
  t=t+DT
  Pin=Pres

In [45]:
Pexplicita

array([[2000.        , 1000.99966462,  502.16567878,    0.        ],
       [2000.        , 1000.99966464,  502.16566551,    0.        ],
       [2000.        , 1000.99966467,  502.16565225,    0.        ],
       ...,
       [2000.        , 1000.99999989,  502.00005512,    0.        ],
       [2000.        , 1000.99999989,  502.00005512,    0.        ],
       [2000.        , 1000.99999989,  502.00005512,    0.        ]])

In [52]:
#Esquema Implicito
Pimplicita=np.zeros((Tiempo//DT,L//dx)) #matriz resultados
Pin=np.ones(L//dx)
Pres=np.zeros(L//dx)
Pin=Pin*Pyac
Pin[0]=Piz
Pin[len(Pin)-1]=Pder
N=L//dx
Matriz_resolucion=np.zeros([N-2,N-2]) #matriz de resolucion

In [53]:
for i in range(N-2): #matriz solucion
  Matriz_resolucion[i,i]=1+2*Eta #porque diagonal fila i es igual a columna
  if i>0: #valores debajo de la diagonal
    Matriz_resolucion[i,i-1]=-Eta
  if i<(N-3): #valores sobre la diagonal
    Matriz_resolucion[i,i+1]=-Eta 

In [54]:
Pcal=np.zeros([N-2]) #Presiones calculadas
Pcal=np.linalg.solve(Matriz_resolucion,Pin[1:-1])

In [55]:
Pcal

array([999.9600016, 999.9600016])

In [56]:
Pres=np.concatenate(([Piz], Pcal, [Pder])) #condiciones de frontera y calculadas
Pres

array([2000.       ,  999.9600016,  999.9600016,    0.       ])

In [57]:
Pimplicita[0]=Pin #presiones con metodo implicito (matriz resultados)
t=DT
while t<Tiempo:
  Pcal=np.linealg.solve(Matriz_resolucion,Pin[1:-1]) #vector calculado
  Pres=np.concatenate(([Piz],Pcal,[Pder])) #vector calc con frontera
  Pimplicita[t//DT]=Pres
  Pin=Pres #presion de resultados o matriz de resultados
  t=t+DT


AttributeError: ignored